# 高德API测试

## 搜索地址

- 代码: `utils/gaode_locations.py`
- 使用高德API搜索"厦门市""鹭燕"共产生112个地址
- 如果缺失其他门店/仓库位置可以直接在高德地图上搜索,添加进`gaode/locations.csv` (需要把物流中心地址放在首行)
- 其中`location`列表示经纬度, 为路径搜索的必须数据

In [1]:
import pandas as pd

locations = pd.read_csv("./gaode/locations.csv")
locations

,Column1,parent,address,distance,pname,importance,biz_ext,biz_type,cityname,type,...,typecode,shopinfo,poiweight,childtype,adname,name,location,tel,shopid,id
0,111,[],灌口镇灌南工业区安仁大道1599号,[],福建省,[],[],[],厦门市,公司企业;公司;医药公司,...,170203,0,[],[],集美区,厦门鹭燕制药有限公司,"118.007317,24.598948",0592-8129302,[],B025004PHK
1,0,[],天英二里26、27号,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],集美区,鹭燕大药房(浒井分店),"118.101334,24.604923",0592-6036378,[],B0FFFWA0TG
2,1,B025003MZA,SM城市广场负一层056号,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],201,湖里区,鹭燕大药房(SM二分店),"118.127344,24.500222",0592-5597622,[],B0FFF3DASO
3,2,[],诚毅北大街61号110,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],集美区,鹭燕大药房(软三分店),"118.044874,24.614532",0592-6683686,[],B0FFKPZJLA
4,3,[],岑西路125号202店面,[],福建省,[],"{'cost': [], 'rating': []}",[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],集美区,鹭燕大药房(集美岑西分店),"118.095742,24.573115",0592-5731139,[],B0FFFS6BYL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,106,[],马巷镇巷南路54号随缘百货对面,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],翔安区,燕鹭大药房(巷南路店),"118.253242,24.662151",[],[],B0G0ASU1J6
108,107,[],马巷镇民安路60号,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],翔安区,鹭燕大药房(马巷分店),"118.250164,24.659526",0592-7620280,[],B02500TG8Y
109,108,B02500QFHK,大同路288-290号,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],202,思明区,鹭燕大药房(思北大同分店),"118.080348,24.456965",0592-2100551,[],B02500S7YU
110,109,[],祥平西路223、225、227号,[],福建省,[],[],[],厦门市,医疗保健服务;医药保健销售店;药房,...,90601,0,[],[],同安区,鹭燕大药房(同安分店),"118.147349,24.727507",0592-7360028,[],B02500QKZK


## 搜索路径

- 代码: `utils/gaode_travel_time.py`
- 使用普通汽车的路径规划api, 结果与货车路径规划可能不一致
- ~~demo只搜索了前10家门店的路径~~
- 假设A到B与B到A的时间一致
- 如果需要完整112个地点路径规划需要3个个人API(5000次/日), 如果需要不同时间段路径时间则需要的API数量更多
- 企业API为300000次/日, 货车路径规划需另付费
- ~~完整路径搜索需要重构代码, 如选择scrapy分布式爬虫~~


In [2]:
import numpy as np

# 时间矩阵(分钟)
distance_matrix = np.genfromtxt('./gaode/distance_matrix_20221227_1657.csv', delimiter=',', dtype=int)
distance_matrix

array([[ 0, 24, 26, ..., 32, 32, 34],
       [24,  0, 26, ..., 32, 27, 25],
       [26, 26,  0, ..., 22, 40, 18],
       ...,
       [32, 32, 22, ...,  0, 57, 29],
       [32, 27, 40, ..., 57,  0, 39],
       [34, 25, 18, ..., 29, 39,  0]])

In [3]:
# 具体路线
trip_details = pd.read_csv('./gaode/trip_details_20221227_1657.csv')
trip_details # 这里的duration以秒为单位, origin与destination为locations.csv的地点顺序

,distance,duration,strategy,tolls,toll_distance,steps,restriction,traffic_lights,origin,destination
0,24668,2181,速度最快,0,0,"[{'instruction': '向东北行驶109米右转', 'orientation':...",0,11,0,8
1,20241,1535,速度最快,0,0,"[{'instruction': '向东北行驶109米右转', 'orientation':...",0,4,0,2
2,24544,2008,速度最快,0,0,"[{'instruction': '向东北行驶109米右转', 'orientation':...",0,6,0,5
3,21307,2061,速度最快,0,0,"[{'instruction': '向东北行驶109米右转', 'orientation':...",0,9,0,7
4,16068,1407,速度最快,5,9349,"[{'instruction': '向东北行驶109米右转', 'orientation':...",0,4,0,1
...,...,...,...,...,...,...,...,...,...,...
6211,23183,2223,速度最快,5,10469,"[{'instruction': '沿民安路向西北行驶21米向右前方行驶', 'orient...",0,5,108,110
6212,23171,2450,速度最快,0,0,"[{'instruction': '沿民安路向东南行驶235米右转', 'orientati...",0,11,108,111
6213,36102,3369,速度最快,0,0,"[{'instruction': '沿大同路向东行驶293米向右前方行驶', 'orient...",0,11,109,110
6214,16373,1703,速度最快,0,0,"[{'instruction': '沿大同路向东行驶293米向右前方行驶', 'orient...",0,11,109,111
